In [1]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [2]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [3]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [4]:
##GPU 사용 시
device = torch.device("cuda:0")

In [5]:
bertmodel, vocab = get_pytorch_kobert_model()

using cached model
using cached model


In [6]:
dataset_train = nlp.data.TSVDataset("ratings_train.txt?dl=1", field_indices=[1,2], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset("ratings_test.txt?dl=1", field_indices=[1,2], num_discard_samples=1)

In [7]:
dataset_train

In [8]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [9]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


In [10]:
## Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [11]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [12]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [13]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device), return_dict=False)# <-- return_dict=False 추가했습니다
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [14]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [15]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [16]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [17]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [18]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [19]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [20]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/home/nlplab/anaconda3/envs/momoKoBERT/lib/python3.6/site-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/2344 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 0.7446473836898804 train acc 0.453125
epoch 1 train acc 0.453125
epoch 1 batch id 201 loss 0.503562867641449 train acc 0.5498289800995025
epoch 1 train acc 0.5498289800995025
epoch 1 batch id 401 loss 0.40499842166900635 train acc 0.6698488154613467
epoch 1 train acc 0.6698488154613467
epoch 1 batch id 601 loss 0.3958743214607239 train acc 0.7235076955074875
epoch 1 train acc 0.7235076955074875
epoch 1 batch id 801 loss 0.40475329756736755 train acc 0.754603620474407
epoch 1 train acc 0.754603620474407
epoch 1 batch id 1001 loss 0.27074697613716125 train acc 0.7739760239760239
epoch 1 train acc 0.7739760239760239
epoch 1 batch id 1201 loss 0.3654276430606842 train acc 0.7872085761865112
epoch 1 train acc 0.7872085761865112
epoch 1 batch id 1401 loss 0.29545465111732483 train acc 0.7969196109921485
epoch 1 train acc 0.7969196109921485
epoch 1 batch id 1601 loss 0.38534975051879883 train acc 0.8057659275452842
epoch 1 train acc 0.8057659275452842
epoch 1 batch id 

  0%|          | 0/782 [00:00<?, ?it/s]

epoch 1 test acc 0.8824928069053708


...

epoch 4 test acc 0.8975583439897699


  0%|          | 0/2344 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.38297325372695923 train acc 0.875
epoch 5 train acc 0.875
epoch 5 batch id 201 loss 0.050985872745513916 train acc 0.9743470149253731
epoch 5 train acc 0.9743470149253731
epoch 5 batch id 401 loss 0.05175464227795601 train acc 0.9754909600997507
epoch 5 train acc 0.9754909600997507
epoch 5 batch id 601 loss 0.26742425560951233 train acc 0.9756135607321131
epoch 5 train acc 0.9756135607321131
epoch 5 batch id 801 loss 0.10472835600376129 train acc 0.9765332397003745
epoch 5 train acc 0.9765332397003745
epoch 5 batch id 1001 loss 0.005985351745039225 train acc 0.9771634615384616
epoch 5 train acc 0.9771634615384616
epoch 5 batch id 1201 loss 0.04447946324944496 train acc 0.9773235845129059
epoch 5 train acc 0.9773235845129059
epoch 5 batch id 1401 loss 0.014704975299537182 train acc 0.9774379907209136
epoch 5 train acc 0.9774379907209136
epoch 5 batch id 1601 loss 0.012505337595939636 train acc 0.9778556371018113
epoch 5 train acc 0.9778556371018113
epoch 5 batc

  0%|          | 0/782 [00:00<?, ?it/s]

epoch 5 test acc 0.8985174232736572


#todo

다른 데이터셋들도 돌려보고 버트 이해하기